In [143]:
import numpy as np
import pickle

with open('TTT_model.pkl','rb') as f:
    model = pickle.load(f)

    

In [144]:
model.keys()

dict_keys(['states', 'connections', 'valuetable'])

In [145]:
states = model['states']
connections = model['connections']
V = model['valuetable']

In [146]:
def identify_state(board):
    turn = np.sum(np.array(board) != 0)
    state_index = states[turn].index(board)
    return turn, state_index

def get_best_action(turn, state_index):
    actions = connections[turn][state_index]
    v_arr = [V[turn + 1][a] for a in actions]
    print(v_arr)
    if turn%2==0:    
        a = np.argmax(v_arr)
    else: 
        a = np.argmin(v_arr)
    best_action = actions[a] 
    next_state = states[turn + 1][best_action]
    return next_state

def translate_action(board_state, next_state):
    for i, b , n in zip(range(9), board_state, next_state):
        if b != n:
            return i

v_states = [
            [1,1,1,0,0,0,0,0,0],
            [0,0,0,1,1,1,0,0,0],
            [0,0,0,0,0,0,1,1,1],
            [1,0,0,1,0,0,1,0,0],
            [0,1,0,0,1,0,0,1,0],
            [0,0,1,0,0,1,0,0,1],
            [1,0,0,0,1,0,0,0,1],
            [0,0,1,0,1,0,1,0,0],
            ]

def check_victory(s):
    # s is a length 9 array
    x = s == 1
    o = s == 2
    sx = v_states @ x
    so = v_states @ o
    if sx.max() == 3:
        return 1
    elif so.max() == 3:
        return -1
    else:
        return 0

gamma = 0.5
def calc_V(turn, i, verbose = False):

    actions = connections[turn][i] 
    V_new = []
    for a in actions:
        s = states[turn + 1][a]
        reward = check_victory(np.array(s))
        if reward == 0 and turn != 8:
            V_new.append( gamma * V[turn + 1][a])

        else:
            V_new.append(reward)
    
    if verbose == True:
        print(V_new)

    print(V_new)
    if turn%2==0:    
        V_max = max(V_new)
        a = np.argmax(V_new)
    else: 
        V_max = min(V_new)
        a = np.argmin(V_new)

    next_state = states[turn + 1][actions[a]]
    return next_state



In [147]:
board_state = [0] * 9
board_state[2] = 1
board_state[8] = 2
print(board_state)
print(np.array(board_state) != 0)
turn, state_index = identify_state(board_state)
print(state_index)

[0, 0, 1, 0, 0, 0, 0, 0, 2]
[False False  True False False False False False  True]
23


In [148]:
import tkinter as tk
import random

v_states = [
            [1,1,1,0,0,0,0,0,0],
            [0,0,0,1,1,1,0,0,0],
            [0,0,0,0,0,0,1,1,1],
            [1,0,0,1,0,0,1,0,0],
            [0,1,0,0,1,0,0,1,0],
            [0,0,1,0,0,1,0,0,1],
            [1,0,0,0,1,0,0,0,1],
            [0,0,1,0,1,0,1,0,0],
            ]

def check_winner(board_state):
    # Mögliche Gewinnkombinationen im TicTacToe
    winning_combinations = [
        (0, 1, 2), (3, 4, 5), (6, 7, 8),  # Reihen
        (0, 3, 6), (1, 4, 7), (2, 5, 8),  # Spalten
        (0, 4, 8), (2, 4, 6)              # Diagonalen
    ]
    for combo in winning_combinations:
        if board_state[combo[0]] == board_state[combo[1]] == board_state[combo[2]] != 0:
            return board_state[combo[0]]  # Gibt 1 zurück für Spieler, 2 für Computer
    return 0  # Kein Gewinner

def on_button_click(button, index, board_state, buttons):
    if board_state[index] == 0:
        button.config(text="X")
        board_state[index] = 1

        winner = check_winner(board_state)
        if winner:
            print("Spieler gewinnt!") if winner == 1 else print("Computer gewinnt!")
            return

        # Pause von 0.5 Sekunden, bevor der Computer seinen Zug macht
        button.after(500, lambda: make_computer_move(board_state, buttons))

def calc_computer_action(board_state):
    turn, state_index = identify_state(board_state)
    next_state = get_best_action(turn, state_index)
    next_move = translate_action(board_state, next_state)
    return next_move

def make_computer_move(board_state, buttons):
    free_indices = [i for i, val in enumerate(board_state) if val == 0]
    if free_indices:
        computer_choice = calc_computer_action(board_state)
        #computer_choice = random.choice(free_indices)
        buttons[computer_choice].config(text="O")
        board_state[computer_choice] = 2

        winner = check_winner(board_state)
        if winner:
            print("Spieler gewinnt!") if winner == 1 else print("Computer gewinnt!")

def create_tictactoe_board():
    root = tk.Tk()
    root.title("TicTacToe Board")

    button_size = 5
    board_state = [0] * 9
    buttons = []

    for index in range(9):
        btn = tk.Button(root, text="-", width=button_size, height=button_size)
        btn.config(command=lambda b=btn, i=index: on_button_click(b, i, board_state, buttons))
        btn.grid(row=index//3, column=index%3, padx=0, pady=0)
        buttons.append(btn)

    root.mainloop()

create_tictactoe_board()

[0.0625, 0.0625, 0.0625, 0.0, 0.0625, 0.0625, 0.0625, 0.0625]
[0.0, 0.25, 0.0, 0.0, 0.25, 0.0]
[1.0, 1.0, 1.0, 0.0]
[1.0, 0.0]


In [189]:
import tkinter as tk
import random

def check_winner(board_state):
    winning_combinations = [
        (0, 1, 2), (3, 4, 5), (6, 7, 8),
        (0, 3, 6), (1, 4, 7), (2, 5, 8),
        (0, 4, 8), (2, 4, 6)
    ]
    for combo in winning_combinations:
        if board_state[combo[0]] == board_state[combo[1]] == board_state[combo[2]] != 0:
            return board_state[combo[0]]
    return 0

def on_button_click(button, index, board_state, buttons):
    if board_state[index] == 0:
        button.config(text="X")
        board_state[index] = 1

        winner = check_winner(board_state)
        if winner:
            print("Spieler gewinnt!") if winner == 1 else print("Computer gewinnt!")
            return

        button.after(500, lambda: make_computer_move(board_state, buttons))

def calc_computer_action(board_state):
    turn, state_index = identify_state(board_state)
    next_state = calc_V(turn, state_index)
    next_move = translate_action(board_state, next_state)
    return next_move

def make_computer_move(board_state, buttons):
    free_indices = [i for i, val in enumerate(board_state) if val == 0]
    if free_indices:
        computer_choice = calc_computer_action(board_state)
        buttons[computer_choice].config(text="O")
        board_state[computer_choice] = 2

        winner = check_winner(board_state)
        if winner:
            print("Spieler gewinnt!") if winner == 1 else print("Computer gewinnt!")

def start_game(starting_player, board_state, buttons):
    if starting_player == "Computer":
        make_computer_move(board_state, buttons)

def choose_starting_player(root, board_state, buttons):
    choice_window = tk.Toplevel(root)
    choice_window.title("Wer beginnt?")
    
    tk.Label(choice_window, text="Wähle den Startspieler:").pack()

    tk.Button(choice_window, text="Spieler", command=lambda: (choice_window.destroy(), start_game("Spieler", board_state, buttons))).pack()
    tk.Button(choice_window, text="Computer", command=lambda: (choice_window.destroy(), start_game("Computer", board_state, buttons))).pack()

def create_tictactoe_board():
    root = tk.Tk()
    root.title("TicTacToe Board")

    button_size = 5
    board_state = [0] * 9
    buttons = []

    for index in range(9):
        btn = tk.Button(root, text="-", width=button_size, height=button_size)
        btn.config(command=lambda b=btn, i=index: on_button_click(b, i, board_state, buttons))
        btn.grid(row=index//3, column=index%3, padx=0, pady=0)
        buttons.append(btn)

    choose_starting_player(root, board_state, buttons)
    root.mainloop()

create_tictactoe_board()

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, -0.03125, -0.03125, -0.03125, 0.0]
[0.25, 1, -0.5, -0.5, -0.5]


KeyboardInterrupt: 